# **저시력자를 위한 원화 화폐 분류**
---
- 본 과제는 UltraLytics YOLO v5 모델 사용을 권장합니다.
    - 본 파일의 목차는 UltraLytics YOLO v5에 맞게 작성되어 있습니다.
    - 다른 모델을 찾아서 사용하셔도 좋습니다.
    - 산출물이 잘 나오면 됩니다 : )
---

## 0.미션
---
- **과제 수행 목표**
    - 본 과제는 Object Detection 문제입니다.
    - Object Detection 문제로 접근하기 위해 **데이터셋 전처리**를 하셔야 합니다.
    - 데이터셋 : money_dataset.zip
        1. 데이터셋은 압축 파일로 제공됩니다.
        2. 압축 파일 안에는 화폐마다 폴더가 개별적으로 존재합니다.
        3. 폴더 안에는 화폐 이미지와 화폐 정보가 담긴 json 파일이 있습니다.
    - 여러분이 직접 촬영한 화폐 사진들을 탐지 과정에서 이용 해보세요.
    - 이미지에 화폐 하나만 나오게 촬영하는 것은 지양해주세요.
    - 다양한 방법으로 화폐를 촬영하고 결과를 확인해보세요.
        - ex 1) 화폐의 모든 종류를 한 이미지에 나오게 촬영
        - ex 2) 여러 화폐를 겹치게 하여 촬영
---
- **Key Point**
    1. 모델에 맞는 폴더 구조 확인
    2. 이미지 축소 비율에 맞춰 좌표값 변경
        - 좌표를 이미지 리사이즈한 비율로 변경
    3. 모델에 맞는 정보 추출/형식 변경
        - json 파일에서 정보 추출 및 모델 형식에 맞게 변경
    4. 화폐당 하나의 클래스로 변경
        - 총 8개 클래스
    5. 모델 선택 필요
---

## 1.환경설정

### (1) 구글 드라이브 연동
---
- 아래의 코드 셀을 반드시 실행시켜야 합니다.
---

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


### (2) 데이터셋 불러오기
---
- **세부요구사항**
    - 데이터셋 파일의 압축을 해제하세요.
---
- 예제 코드에서는 zipfile 모듈을 이용하였습니다.
    - [zipfile document](https://docs.python.org/3/library/zipfile.html#zipfile-objects)
    - 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
---

In [ ]:
import zipfile

In [ ]:
# 데이터셋 압축 파일 경로 : 유저별로 상이할 수 있음
money_data = zipfile.ZipFile('/content/drive/MyDrive/money_dataset.zip')

In [ ]:
# 데이터셋 압축 해제
money_data.extractall('/content/drive/MyDrive/0323Dataset/')

## 2.데이터 전처리

### (1) 폴더 구조 생성 및 파일 이동
---
- **세부요구사항**
    -  모델에서 요구하는 폴더 구조를 만들어야 합니다.
        - Hint : Image와 Label을 구분하는 폴더를 만들어 주세요
---
- 예제 코드에서는 glob, shutil 모듈을 이용하였습니다.
    - [glob document](https://docs.python.org/3/library/glob.html) | [shutil document](https://docs.python.org/3/library/shutil.html)
    - 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
---

In [ ]:
# 1.폴더 구조 만들기
!mkdir /content/drive/MyDrive/0323Dataset/images;
!mkdir /content/drive/MyDrive/0323Dataset/images/train
!mkdir /content/drive/MyDrive/0323Dataset/images/val

!mkdir /content/drive/MyDrive/0323Dataset/labels;
!mkdir /content/drive/MyDrive/0323Dataset/labels/train; mkdir /content/drive/MyDrive/0323Dataset/labels/val

mkdir: cannot create directory ‘/content/drive/MyDrive/0323Dataset/images’: File exists
mkdir: cannot create directory ‘/content/drive/MyDrive/0323Dataset/images/train’: File exists
mkdir: cannot create directory ‘/content/drive/MyDrive/0323Dataset/images/val’: File exists
mkdir: cannot create directory ‘/content/drive/MyDrive/0323Dataset/labels’: File exists
mkdir: cannot create directory ‘/content/drive/MyDrive/0323Dataset/labels/train’: File exists
mkdir: cannot create directory ‘/content/drive/MyDrive/0323Dataset/labels/val’: File exists


In [ ]:
import glob, shutil

In [ ]:
# 2. Dataset metadata 입력
won_list = ['10', '50', '100', '500', '1000', '5000', '10000', '50000']
data_path = '/content/drive/MyDrive/0323Dataset/'

---
- 데이터를 Training set | Validation set으로 분할하세요.
    - 예시 : Training과 Validation은 8:2로 분리
- Hint : 이미지 데이터는 /images에, JSON 데이터는 /labels에 넣어주세요
    - 예시 : /dataset/images/train, /dataset/labels/train
    - 예제 코드에서는 glob, shutil 모듈을 이용하였습니다.
    - [glob document](https://docs.python.org/3/library/glob.html) | [shutil document](https://docs.python.org/3/library/shutil.html)

    ※ 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
    
---

In [ ]:
import os

In [ ]:
########################
# 이 셀부터 코드 작성하세요
########################
# 3. 데이터를 Training set | Validation set으로 분할하세요.
num_10 = int( ( len(os.listdir(data_path + '10')) / 2 ) )
num_50 = int( ( len(os.listdir(data_path + '50')) / 2 ) )
num_100 = int( ( len(os.listdir(data_path + '100')) / 2 ) )
num_500 = int( ( len(os.listdir(data_path + '500')) / 2 ) )
num_1000 = int( ( len(os.listdir(data_path + '1000')) / 2 ) )
num_5000 = int( ( len(os.listdir(data_path + '5000')) / 2 ) )
num_10000 = int( ( len(os.listdir(data_path + '10000')) / 2 ) )
num_50000 = int( ( len(os.listdir(data_path + '50000')) / 2 ) )

In [ ]:
print (num_10, num_50, num_100, num_500, num_1000, num_5000, num_10000, num_50000)

0 0 0 0 0 0 0 0


In [ ]:
images_10 = glob.glob(f"{data_path+'10'}/*.jpg")
images_50 = glob.glob(f"{data_path+'50'}/*.jpg")
images_100 = glob.glob(f"{data_path+'100'}/*.jpg")
images_500 = glob.glob(f"{data_path+'500'}/*.jpg")
images_1000 = glob.glob(f"{data_path+'1000'}/*.jpg")
images_5000 = glob.glob(f"{data_path+'5000'}/*.jpg")
images_10000 = glob.glob(f"{data_path+'10000'}/*.jpg")
images_50000 = glob.glob(f"{data_path+'50000'}/*.jpg")
json_10 = glob.glob(f"{data_path+'10'}/*.json")
json_50 = glob.glob(f"{data_path+'50'}/*.json")
json_100 = glob.glob(f"{data_path+'100'}/*.json")
json_500 = glob.glob(f"{data_path+'500'}/*.json")
json_1000 = glob.glob(f"{data_path+'1000'}/*.json")
json_5000 = glob.glob(f"{data_path+'5000'}/*.json")
json_10000 = glob.glob(f"{data_path+'10000'}/*.json")
json_50000 = glob.glob(f"{data_path+'50000'}/*.json")

images_or_10 = glob.glob(f"{data_path+'10'}/*.jpg")
images_or_50 = glob.glob(f"{data_path+'50'}/*.jpg")
images_or_100 = glob.glob(f"{data_path+'100'}/*.jpg")
images_or_500 = glob.glob(f"{data_path+'500'}/*.jpg")
images_or_1000 = glob.glob(f"{data_path+'1000'}/*.jpg")
images_or_5000 = glob.glob(f"{data_path+'5000'}/*.jpg")
images_or_10000 = glob.glob(f"{data_path+'10000'}/*.jpg")
images_or_50000 = glob.glob(f"{data_path+'50000'}/*.jpg")

In [ ]:
images_10[5], json_10[5], images_or_10[5]

IndexError: ignored

In [ ]:
from IPython.display import Image

In [ ]:
Image(filename=data_path + 'images/val/' + img_val[3], width=640)

NameError: ignored

In [ ]:
images_100

In [ ]:
import random

In [ ]:
random.shuffle(images_10)
random.shuffle(images_50)
random.shuffle(images_100)
random.shuffle(images_500)
random.shuffle(images_1000)
random.shuffle(images_5000)
random.shuffle(images_10000)
random.shuffle(images_50000)

In [ ]:
images_train_10, images_valid_10 = [], []
images_train_50, images_valid_50 = [], []
images_train_100, images_valid_100 = [], []
images_train_500, images_valid_500 = [], []
images_train_1000, images_valid_1000 = [], []
images_train_5000, images_valid_5000 = [], []
images_train_10000, images_valid_10000 = [], []
images_train_50000, images_valid_50000 = [], []

json_train_10, json_valid_10 = [], []
json_train_50, json_valid_50 = [], []
json_train_100, json_valid_100 = [], []
json_train_500, json_valid_500 = [], []
json_train_1000, json_valid_1000 = [], []
json_train_5000, json_valid_5000 = [], []
json_train_10000, json_valid_10000 = [], []
json_train_50000, json_valid_50000 = [], []

In [ ]:
for i in range (int(num_10*0.8)):
  images_train_10.append(images_10[i])
for i in range (int(num_50*0.8)):
  images_train_50.append(images_50[i])
for i in range (int(num_100*0.8)):
  images_train_100.append(images_100[i])
for i in range (int(num_500*0.8)):
  images_train_500.append(images_500[i])
for i in range (int(num_1000*0.8)):
  images_train_1000.append(images_1000[i])
for i in range (int(num_5000*0.8)):
  images_train_5000.append(images_5000[i])
for i in range (int(num_10000*0.8)):
  images_train_10000.append(images_10000[i])
for i in range (int(num_50000*0.8)):
  images_train_50000.append(images_50000[i])

In [ ]:
for filename in images_10:
 if filename in images_train_10:
   pass
 else:
   images_valid_10.append(filename)

for filename in images_50:
 if filename in images_train_50:
   pass
 else:
   images_valid_50.append(filename)

for filename in images_100:
 if filename in images_train_100:
   pass
 else:
   images_valid_100.append(filename)

for filename in images_500:
 if filename in images_train_500:
   pass
 else:
   images_valid_500.append(filename)

for filename in images_1000:
 if filename in images_train_1000:
   pass
 else:
   images_valid_1000.append(filename)

for filename in images_5000:
 if filename in images_train_5000:
   pass
 else:
   images_valid_5000.append(filename)

for filename in images_10000:
 if filename in images_train_10000:
   pass
 else:
   images_valid_10000.append(filename)

for filename in images_50000:
 if filename in images_train_50000:
   pass
 else:
   images_valid_50000.append(filename)

In [ ]:
len(images_10), len(images_train_10), len(images_valid_10)
len(images_50), len(images_train_50), len(images_valid_50)
len(images_100), len(images_train_100), len(images_valid_100)
len(images_500), len(images_train_500), len(images_valid_500)
len(images_1000), len(images_train_1000), len(images_valid_1000)
len(images_5000), len(images_train_5000), len(images_valid_5000)
len(images_10000), len(images_train_10000), len(images_valid_10000)
len(images_50000), len(images_train_50000), len(images_valid_50000)

(0, 0, 0)

In [ ]:
for i in range (int(num_10)):
  if images_or_10[i] in images_train_10:
    json_train_10.append(json_10[i])
    shutil.move( images_or_10[i], data_path + 'images/train' )
    shutil.move( json_10[i], data_path + 'labels/train' )
  else:
    json_valid_10.append(json_10[i])
    shutil.move( images_or_10[i], data_path + 'images/val' )
    shutil.move( json_10[i], data_path + 'labels/val' )

for i in range (int(num_50)):
  if images_or_50[i] in images_train_50:
    json_train_50.append(json_50[i])
    shutil.move( images_or_50[i], data_path + 'images/train' )
    shutil.move( json_50[i], data_path + 'labels/train' )
  else:
    json_valid_50.append(json_50[i])
    shutil.move( images_or_50[i], data_path + 'images/val' )
    shutil.move( json_50[i], data_path + 'labels/val' )

for i in range (int(num_100)):
  if images_or_100[i] in images_train_100:
    json_train_100.append(json_100[i])
    shutil.move( images_or_100[i], data_path + 'images/train' )
    shutil.move( json_100[i], data_path + 'labels/train' )
  else:
    json_valid_100.append(json_100[i])
    shutil.move( images_or_100[i], data_path + 'images/val' )
    shutil.move( json_100[i], data_path + 'labels/val' )

for i in range (int(num_500)):
  if images_or_500[i] in images_train_500:
    json_train_500.append(json_500[i])
    shutil.move( images_or_500[i], data_path + 'images/train' )
    shutil.move( json_500[i], data_path + 'labels/train' )
  else:
    json_valid_500.append(json_500[i])
    shutil.move( images_or_500[i], data_path + 'images/val' )
    shutil.move( json_500[i], data_path + 'labels/val' )

for i in range (int(num_1000)):
  if images_or_1000[i] in images_train_1000:
    json_train_1000.append(json_1000[i])
    shutil.move( images_or_1000[i], data_path + 'images/train' )
    shutil.move( json_1000[i], data_path + 'labels/train' )
  else:
    json_valid_1000.append(json_1000[i])
    shutil.move( images_or_1000[i], data_path + 'images/val' )
    shutil.move( json_1000[i], data_path + 'labels/val' )

for i in range (int(num_5000)):
  if images_or_5000[i] in images_train_5000:
    json_train_5000.append(json_5000[i])
    shutil.move( images_or_5000[i], data_path + 'images/train' )
    shutil.move( json_5000[i], data_path + 'labels/train' )
  else:
    json_valid_5000.append(json_5000[i])
    shutil.move( images_or_5000[i], data_path + 'images/val' )
    shutil.move( json_5000[i], data_path + 'labels/val' )

for i in range (int(num_10000)):
  if images_or_10000[i] in images_train_10000:
    json_train_10000.append(json_10000[i])
    shutil.move( images_or_10000[i], data_path + 'images/train' )
    shutil.move( json_10000[i], data_path + 'labels/train' )
  else:
    json_valid_10000.append(json_10000[i])
    shutil.move( images_or_10000[i], data_path + 'images/val' )
    shutil.move( json_10000[i], data_path + 'labels/val' )

for i in range (int(num_50000)):
  if images_or_50000[i] in images_train_50000:
    json_train_50000.append(json_50000[i])
    shutil.move( images_or_50000[i], data_path + 'images/train' )
    shutil.move( json_50000[i], data_path + 'labels/train' )
  else:
    json_valid_50000.append(json_50000[i])
    shutil.move( images_or_50000[i], data_path + 'images/val' )
    shutil.move( json_50000[i], data_path + 'labels/val' )


In [ ]:
len(json_valid_10), len(images_valid_10), len(json_valid_50), len(images_valid_50), len(json_valid_100), len(images_valid_100), len(json_valid_500), len(images_valid_500), len(json_valid_1000), len(images_valid_1000), len(json_valid_5000), len(images_valid_5000), len(json_valid_10000), len(images_valid_10000), len(json_valid_50000), len(images_valid_50000)

(88, 88, 88, 88, 88, 88, 88, 88, 172, 172, 174, 174, 174, 174, 174, 174)

In [ ]:
len(os.listdir(data_path + 'images/train')), len(os.listdir(data_path + 'images/val')), len(os.listdir(data_path + 'labels/train')), len(os.listdir(data_path + 'labels/val'))

(4172, 1046, 4172, 1046)

아마도 홀수개로 나누는게있어서?
4175 1043

### (2) json에서 정보 추출
---
- **세부요구사항**
    - json 파일에서 필요한 정보를 추출하세요:
        - 위치 정보 : x1, x2, y1, y2
        - 박스 정보 : shape_type
        - 클래스 정보 : labels
    - 화폐당 하나의 클래스로 변경하세요.
        - json 파일에는 화폐 클래스가 앞뒷면으로 구분되어 있습니다.
        - 화폐의 앞뒷면 구분을 없애주세요.
            - 예시 : 'ten_front', 'ten_back' -> 'ten'
    - 화폐의 위치 정보를 YOLO 모델 형식에 맞게 변경 해주세요.
        - 사용되는 이미지는 원본에서 1/5로 축소되어 있습니다.
        - json 파일의 정보는 원본 기준 데이터이므로 위치 정보 추출을 할 때 x값과 y값을 1/5로 줄여주세요.
    - 이렇게 변경된 정보를 YOLO label 형식에 맞게 txt파일로 저장 해 주세요.
        - Hint : YOLO Labeling Format [label, x-center, y-center, width-norm, height-norm]
---

In [ ]:
import os, json

In [ ]:
json_path = data_path +'labels/'
temp_list = ['train', 'val']

In [ ]:
json_train = os.listdir(data_path + 'labels/train/')
json_val = os.listdir(data_path + 'labels/val/')

In [ ]:
json_val[1]

'10_559_1.json'

In [ ]:
json_train_or=[]
json_val_or=[]

In [ ]:
json_train_ch=[]
json_val_ch=[]

In [ ]:
########################
# Json 파일에서 필요한 정보만 골라 txt로 바꾸는 작업임을 기억하세요!
########################


# 나중에 10000_B_DESK_0_124
for i in range (int(len(json_train)) ):
  with open(data_path + 'labels/train/' + json_train[i]) as x:
    data = json.load(x)
    json_train_or.append(data)
    data = 0

for i in range (int(len(json_val)) ):
  with open(data_path + 'labels/val/' + json_val[i]) as x:
    data = json.load(x)
    json_val_or.append(data)
    data = 0

In [ ]:
len(glob.glob('/content/drive/MyDrive/0323Dataset/labels/train/*')), len(glob.glob('/content/drive/MyDrive/0323Dataset/labels/val/*'))

(4172, 1046)

In [ ]:
json_val_or[13]

{'version': '3.16.7',
 'flags': {},
 'shapes': [{'label': 'Ten_front',
   'line_color': None,
   'fill_color': None,
   'points': [[787.6509076799999, 1549.5320736], [1469.0110428, 2226.476448]],
   'shape_type': 'rectangle',
   'flags': {}}],
 'lineColor': [0, 255, 0, 128],
 'fillColor': [255, 0, 0, 128],
 'imagePath': '10_648_1.jpg',
 'imageWidth': 2268,
 'imageHeight': 4032,
 'imageData': None}

### (3) 데이터셋 정보가 담긴 파일 생성
---
- **세부요구사항**
    - 파일 안에 있어야 할 정보는 아래와 같습니다.
        - 학습할 클래스 이름 정보
        - 학습할 클래스 수 정보
        - Training, Validation 데이터셋 위치 정보
---
- 가장 대중적으로 이용하는 라이브러리는 yaml 입니다.
    - [yaml document](https://pyyaml.org/wiki/PyYAMLDocumentation)
    - 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
---

In [ ]:
import yaml

In [ ]:
won_dict = {0:'10', 1:'50', 2:'100', 3:'500', 4:'1000', 5:'5000', 6:'10000', 7:'50000'}

In [ ]:
# 언젠가 우리는 일일이 손으로 쓰지 못할 만큼 어마무시한 클래스의 숫자를 가진진 다중 분류 문제를 만날 수 있습니다.
# 그러니 f-string을 통해 코딩하는 습관을 기르도록 합시다.

document = f'''
path: /content/Dataset
train: images/train
val: images/val

nc: {len(won_list)}
names: {won_list}

'''
with open('/content/Dataset/money.yaml', 'w') as f:
    f.write(document)
    f.close()

    #호준님 코드

FileNotFoundError: ignored

## 3.모델링

### (1) 모델 라이브러리 설치
---

In [ ]:
!pip install jedi

In [ ]:
!git clone https://github.com/ultralytics/yolov5

In [ ]:
## yolov5 폴더 requirements.txt 수정 필요
## setuptools<=64.0.2

temp_str = 'setuptools<=64.0.2\n'

f = open('/content/yolov5/requirements.txt', 'r')
f_str = f.readlines()
f.close()

f2 = open('/content/yolov5/requirements.txt', 'w')

for idx, val in enumerate(f_str) :
    if 'setuptools' in val :
        idx_v = idx
        f_str.remove(val)
        f_str.insert(idx_v, temp_str)

for val in f_str :
    f2.write(val)

f2.close()

In [ ]:
!cd yolov5; pip install -r requirements.txt

### (2) 가중치 파일 다운로드
---
- **세부요구사항**
    - 모델 개발자가 제공하는 사전 학습 가중치 파일을 다운로드 하세요.
        - 해당 과정이 불필요하다면 넘어가셔도 됩니다!
---

In [ ]:
!mkdir /content/yolov5/pretrained

In [ ]:
!wget -O /content/yolov5/pretrained/yolov5s.pt https://github.com/ultralytics/yolov5/releases/download/v7.0/yolov5l-seg.pt

### (3) 학습 : train.py
---
- **세부요구사항**
    - UltraLytics YOLO v5에는 아래의 데이터가 필요합니다.
        - 데이터셋 정보가 담긴 yaml 파일
        - 사용하려는 모델 구조에 대한 yaml 파일
        - 사용하려는 모델의 가중치 파일
---

In [ ]:
!cd yolov5; python train.py \
  --data '/content/drive/MyDrive/Datasets2/moneyclass.yaml' \
  --cfg '/content/yolov5/models/yolov5l.yaml' \
  --weights  '/content/drive/MyDrive/Datasets2/yolov5l.pt' \
  --epochs 1000 \
  --patience 5 \
  --img 320 \
  --project 'trained' \
  --name 'train_money' \
  --exist-ok
# --device cpu

## 4.탐지 : detect.py
---
- **세부요구사항**
    - 학습 과정에서 생성된 가중치 파일을 이용하세요.
    - IoU threshold를 0.25 이하로 설정하세요.
    - confidence threshold를 0.75 이상으로 설정하세요.
---
- 여러분이 **직접 촬영한 화폐 사진과 동영상**을 탐지 과정에 이용하여 결과를 확인하세요.
    - 조건
        1. 화폐의 수를 늘려가며 촬영 해보세요.
            - ex) 50원 하나, 50원 둘, 50원 셋, ...
        2. 화폐의 종류를 늘려가며 촬영 해보세요.
            - ex) 50원 하나와 100원 하나, 50원 하나와 100원 하나와 1000원 하나, ...
        3. 사진은 최소 30장 이상, 동영상은 최소 하나 이상 촬영하여 사용 해보세요.
---

In [ ]:
!mkdir /content/drive/MyDrive/Datasets2/images/test;

In [ ]:
dataset_path  = '/content/drive/MyDrive/Datasets2/'

test_path = dataset_path + 'money_picture.zip'

In [ ]:
test_money = zipfile.ZipFile(test_path)

In [ ]:
test_money.extractall(dataset_path+'images/test')

In [ ]:
print('test data:',len(os.listdir(dataset_path+'images/test')))

In [ ]:
!cd yolov5; python detect.py \
    --weights '/content/yolov5/trained/train_money/weights/best.pt' \
    --source '/content/drive/MyDrive/Datasets2/images/test' \
    --project '/content/yolov5/detected' \
    --name 'images' \
    --img 320 \
    --conf-thres 0.75 \
    --iou-thres 0.25 \
    --line-thickness 2 \
    --exist-ok
    # --device CPU

In [ ]:
from IPython.display import Image
from google.colab import files

In [ ]:
test_class = os.listdir('/content/yolov5/detected/images/')